In [15]:
# ==== CONFIG ====
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')
BASE_DIR  = Path("/content/drive/MyDrive/Faiss/sentence_transformers_all_MiniLM_L6_v2")  # 경로 설정
EMB_DIR   = BASE_DIR / "embeddings"      # 원본(.npy/.json)
INDEX_DIR = BASE_DIR / "index"           # 산출물 저장 위치
OVERWRITE = False                        # True면 기존 산출물 덮어씀


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- 원본 embeddings 중복 검토

In [35]:
# ==== CONFIG (네 설정 사용) ====
from pathlib import Path
import json, re
from collections import Counter, defaultdict
import itertools

BASE_DIR  = Path("/content/drive/MyDrive/Faiss/Snowflake_snowflake_arctic_embed_l_v2.0") # 경로 설정
EMB_DIR   = BASE_DIR / "embeddings"      # 원본(.npy/.json)

# 스캔 대상: id_*_BAAI_bge_base_en_cls(.json 포함 가능)
cand_files = sorted(EMB_DIR.glob("id_*_Snowflake_snowflake_arctic_embed_l_v2.0_cls*")) # 모델 이름에 맞게 입력 (버전 다음 부분부터 _cls/_mean까지)

print("=== 스캔 대상 파일 ===")
if not cand_files:
    print("(!) 대상 파일을 찾지 못했습니다. 경로/파일명을 확인하세요.")
for p in cand_files:
    print("-", p.name)

# 예: id_2.4_BAAI_bge_base_en_cls[.json] 에서 2.4만 뽑기
version_rx = re.compile(r"id_(?P<ver>\d+\.\d+)_Snowflake_snowflake_arctic_embed_l_v2.0_cls", re.IGNORECASE)

# 수집 컨테이너
all_rows = []     # (id, file, index, version)
per_file_stats = []
per_file_dup_examples = {}  # {file: {id: [indices,...]}}

# ==== 파일별 검사 (내부 중복 / index 연속성) ====
for path in cand_files:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    m = version_rx.search(path.name)
    version = m.group("ver") if m else "unknown"

    idmap = data.get("id_mapping", [])
    ids   = [row["id"] for row in idmap]
    idxs  = [row["index"] for row in idmap]

    # 내부 중복(id)
    cnt = Counter(ids)
    dup_ids = [k for k, c in cnt.items() if c > 1]

    # 중복 id의 인덱스들 예시 수집
    dup_detail = {}
    if dup_ids:
        pos = defaultdict(list)
        for row in idmap:
            pos[row["id"]].append(row["index"])
        for _id in dup_ids:
            dup_detail[_id] = sorted(pos[_id])
        per_file_dup_examples[path.name] = dup_detail

    # 인덱스 연속성(0..N-1) & 중복 여부
    idx_set = set(idxs)
    idx_is_unique = (len(idx_set) == len(idxs))
    idx_contiguous = (idx_is_unique and len(idx_set) > 0 and min(idx_set) == 0 and max(idx_set) == len(idx_set)-1)

    per_file_stats.append({
        "file": path.name,
        "version": version,
        "items": len(idmap),
        "unique_ids": len(cnt),
        "dup_ids_in_file": len(dup_ids),
        "index_unique": idx_is_unique,
        "index_contiguous_0_to_n_1": idx_contiguous,
    })

    # 전체 행 수집(파일 간 중복 검사용)
    for row in idmap:
        all_rows.append((row["id"], path.name, row["index"], version))

# ==== 파일별 요약 출력 ====
print("\n=== 파일별 기본 통계 ===")
for s in per_file_stats:
    print(f"{s['file']} | ver={s['version']} | items={s['items']:,} | unique_ids={s['unique_ids']:,} "
          f"| dup_in_file={s['dup_ids_in_file']:,} | idx_unique={s['index_unique']} | idx_contiguous={s['index_contiguous_0_to_n_1']}")

# 내부 중복 상세 예시 출력
print("\n=== (옵션) 내부 중복 상세 예시 (파일별 상위 최대 5개 id) ===")
if not per_file_dup_examples:
    print("내부 중복 id 없음.")
else:
    for fname, dupdict in per_file_dup_examples.items():
        print(f"\n[파일] {fname}  (중복 id 개수: {len(dupdict)})")
        for _id, positions in itertools.islice(dupdict.items(), 5):
            print(f"- id={_id} -> indices={positions}")

# ==== 파일 간 중복(id) ====
by_id = defaultdict(list)
for _id, fname, idx, ver in all_rows:
    by_id[_id].append((fname, idx, ver))

cross_file_dups = {}  # id -> [(file, idx, ver), ...] (두 개 이상 다른 파일에 등장)
for _id, occ in by_id.items():
    files = {x[0] for x in occ}
    if len(files) > 1:
        cross_file_dups[_id] = occ

print("\n=== 파일 간 중복 요약 ===")
total_distinct_ids = len(by_id)
num_cross_dup_ids = len(cross_file_dups)
print(f"- 서로 다른 ID 총계: {total_distinct_ids:,}")
print(f"- 파일 간 중복 ID 개수: {num_cross_dup_ids:,}")
if num_cross_dup_ids == 0:
    print("파일 간 중복 없음.")

# 파일 간 중복 예시 출력
if num_cross_dup_ids > 0:
    print("\n=== 파일 간 중복 상세 예시 (상위 10개) ===")
    shown = 0
    for _id, occ in cross_file_dups.items():
        files = sorted({o[0] for o in occ})
        print(f"\n[id] { _id }  (등장 파일 수: {len(files)})")
        print("  files:", ", ".join(files))
        # 각 발생지 최대 5줄 예시
        for f, i, v in itertools.islice(sorted(occ, key=lambda x: (x[0], x[1])), 5):
            print(f"  - {f} | index={i} | ver={v}")
        shown += 1
        if shown >= 10:
            break

# ==== 인덱스 불연속/중복 파일만 골라 보여주기 ====
bad_index_files = [s for s in per_file_stats if not s["index_contiguous_0_to_n_1"]]
print("\n=== 인덱스 불연속/중복 의심 파일 ===")
if not bad_index_files:
    print("없음 (모든 파일의 index가 0..N-1 연속이고 중복 없음).")
else:
    for s in bad_index_files:
        print(f"- {s['file']} | items={s['items']:,} | idx_unique={s['index_unique']} | idx_contiguous={s['index_contiguous_0_to_n_1']}")


=== 스캔 대상 파일 ===
- id_2.4_Snowflake_snowflake_arctic_embed_l_v2.0_cls.json
- id_2.5_Snowflake_snowflake_arctic_embed_l_v2.0_cls.json
- id_2.6_Snowflake_snowflake_arctic_embed_l_v2.0_cls.json
- id_2.7_Snowflake_snowflake_arctic_embed_l_v2.0_cls.json
- id_2.8_Snowflake_snowflake_arctic_embed_l_v2.0_cls.json

=== 파일별 기본 통계 ===
id_2.4_Snowflake_snowflake_arctic_embed_l_v2.0_cls.json | ver=2.4 | items=7,902 | unique_ids=7,899 | dup_in_file=3 | idx_unique=True | idx_contiguous=True
id_2.5_Snowflake_snowflake_arctic_embed_l_v2.0_cls.json | ver=2.5 | items=8,122 | unique_ids=8,119 | dup_in_file=3 | idx_unique=True | idx_contiguous=True
id_2.6_Snowflake_snowflake_arctic_embed_l_v2.0_cls.json | ver=2.6 | items=8,093 | unique_ids=8,090 | dup_in_file=3 | idx_unique=True | idx_contiguous=True
id_2.7_Snowflake_snowflake_arctic_embed_l_v2.0_cls.json | ver=2.7 | items=8,203 | unique_ids=8,200 | dup_in_file=3 | idx_unique=True | idx_contiguous=True
id_2.8_Snowflake_snowflake_arctic_embed_l_v2.0_cls.jso

- index 생성

In [36]:
# ==== CONFIG ====
from pathlib import Path
BASE_DIR   = Path("/content/drive/MyDrive/Faiss/Snowflake_snowflake_arctic_embed_l_v2.0")  # 경로 설정
EMB_DIR    = BASE_DIR / "embeddings"
INDEX_DIR  = BASE_DIR / "index"
OVERWRITE  = False

# 이 줄만 모델별로 바꿔 쓰면 됨
MODEL_NAME = "Snowflake_snowflake_arctic_embed_l_v2.0_cls"   # _cls/_mean까지

# ==== IMPORT (faiss 자동 설치) ====
try:
    import faiss  # noqa
except Exception:
    import sys, subprocess
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "faiss-cpu"], check=True)
    import faiss

import os, glob, json, numpy as np

# ==== FUNCTIONS ====
def ensure_dirs():
    INDEX_DIR.mkdir(parents=True, exist_ok=True)

def key_of(path_str: str) -> str:
    """
    현재 파일명 규칙:
      - embeddings_<ver>_<MODEL_NAME>.npy
      - id_<ver>_<MODEL_NAME>           (확장자 없음 가능)
      - metadata_<ver>_<MODEL_NAME>.parquet
    페어링 키: '<ver>_<MODEL_NAME>'
    """
    name = os.path.basename(path_str)
    for prefix in ("embeddings_", "id_", "metadata_"):
        sig = f"_{MODEL_NAME}"
        if name.startswith(prefix) and sig in name:
            core = name[len(prefix):]
            core = core.replace(".npy", "").replace(".json", "").replace(".parquet", "")
            return core
    return name

def find_pairs(emb_dir: Path):
    emb_glob = f"embeddings_*_{MODEL_NAME}.npy"
    id_json_glob = f"id_*_{MODEL_NAME}.json"
    id_nox_glob  = f"id_*_{MODEL_NAME}"        # 확장자 없는 케이스

    emb_paths = sorted(glob.glob(str(emb_dir / emb_glob)))
    map_paths = sorted(set(
        glob.glob(str(emb_dir / id_json_glob)) +
        glob.glob(str(emb_dir / id_nox_glob))
    ))

    emb_by_key = {key_of(p): p for p in emb_paths}
    map_by_key = {key_of(p): p for p in map_paths}
    keys = sorted(set(emb_by_key) & set(map_by_key))
    if not keys:
        raise RuntimeError(
            "페어링 실패: 파일명을 확인하세요.\n"
            f"- embeddings 패턴: {emb_glob}\n"
            f"- id 패턴: {id_json_glob} / {id_nox_glob}\n"
            f"- EMB_DIR: {emb_dir}\n"
        )
    return emb_by_key, map_by_key, keys

def _load_ids(map_path: str):
    with open(map_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    id_list = data["id_mapping"] if isinstance(data, dict) and "id_mapping" in data else data
    if not isinstance(id_list, list) or not id_list or not isinstance(id_list[0], dict) or "id" not in id_list[0]:
        raise ValueError(f"[형식 오류] {map_path} : 'id' 키를 가진 dict 리스트가 필요합니다.")
    return id_list

def validate_pairs(emb_by_key, map_by_key, keys):
    total_rows, dim = 0, None
    for k in keys:
        Xmm = np.load(emb_by_key[k], mmap_mode="r")
        rows, d = Xmm.shape
        ids = _load_ids(map_by_key[k])
        if rows != len(ids):
            raise ValueError(f"[행수 불일치] {k}: npy={rows} vs ids={len(ids)}")
        if dim is None:
            dim = d
        elif d != dim:
            raise ValueError(f"[차원 불일치] {k}: 기대 {dim}, 실제 {d}")
        total_rows += rows
    return {"total_rows": total_rows, "dimension": dim}

def build_faiss_index(emb_by_key, keys, index_dir: Path, overwrite=False):
    index_path = index_dir / "faiss.index"
    if index_path.exists() and not overwrite:
        return str(index_path), False

    vecs_list, datasets, dim = [], {}, None
    for k in keys:
        X = np.load(emb_by_key[k]).astype("float32")
        vecs_list.append(X)
        datasets[k] = {"rows": int(X.shape[0]), "dim": int(X.shape[1])}
        if dim is None:
            dim = X.shape[1]
    X_all = np.concatenate(vecs_list, axis=0)

    faiss.normalize_L2(X_all)
    index = faiss.IndexFlatIP(dim)
    index.add(X_all)
    faiss.write_index(index, str(index_path))
    return str(index_path), datasets

def build_global_ids_from_mappings(map_by_key: dict, keys: list, index_dir: Path):
    out_path = Path(index_dir) / "global_ids.json"
    global_ids = []
    for k in keys:
        for item in _load_ids(map_by_key[k]):
            global_ids.append(item["id"])
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump({"ids": global_ids}, f, ensure_ascii=False, indent=2)
    return str(out_path), len(global_ids)

def write_stats(index_dir: Path, dimension: int, total_rows: int, datasets: dict, keys=None):
    version_ranges, cursor = [], 0
    if keys:
        for k in keys:
            rows = int(datasets[k]["rows"])
            if rows <= 0:
                continue
            start, end = cursor, cursor + rows - 1
            version_ranges.append({"version": str(k), "start": start, "end": end})
            cursor += rows

    payload = {
        "model": "intfloat/e5-large-v2",  # 스키마 유지(원 코드와 키 동일)
        "pooling": "mean",
        "metric": "cosine (IP on L2-normalized vectors)",
        "dimension": int(dimension),
        "total_rows": int(total_rows),
        "datasets": datasets,
        "version_ranges": version_ranges
    }
    stats_path = Path(index_dir) / "stats.json"
    with open(stats_path, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)
    return str(stats_path)

def verify(index_dir: Path):
    idx = faiss.read_index(str(Path(index_dir) / "faiss.index"))
    with open(Path(index_dir) / "stats.json", encoding="utf-8") as f:
        s = json.load(f)

    assert idx.ntotal == s["total_rows"], "인덱스 행수와 stats 불일치"
    assert idx.d == s["dimension"], "인덱스 차원과 stats 불일치"

    with open(Path(index_dir) / "global_ids.json", encoding="utf-8") as f:
        gids = json.load(f)["ids"]
    assert len(gids) == s["total_rows"], "global_ids 개수와 stats 불일치"

    vr = s.get("version_ranges", [])
    if vr:
        assert vr[-1]["end"] == len(gids) - 1, "version_ranges의 end와 global_ids 길이가 맞지 않습니다."
    return {"ntotal": idx.ntotal, "dimension": idx.d, "datasets": list(s["datasets"].keys()), "version_ranges": vr}

# ==== MAIN ====
def main():
    ensure_dirs()
    emb_by_key, map_by_key, keys = find_pairs(EMB_DIR)
    summary = validate_pairs(emb_by_key, map_by_key, keys)

    index_path, datasets = build_faiss_index(emb_by_key, keys, INDEX_DIR, overwrite=OVERWRITE)
    created = isinstance(datasets, dict)

    if created:
        stats_path = write_stats(INDEX_DIR, summary["dimension"], summary["total_rows"], datasets, keys)
    else:
        stats_path = str(Path(INDEX_DIR) / "stats.json")
        if not Path(stats_path).exists():
            raise RuntimeError("faiss.index는 존재하지만 stats.json이 없습니다. OVERWRITE=True 로 재생성하세요.")

    gid_path, gid_len = build_global_ids_from_mappings(map_by_key, keys, INDEX_DIR)

    check = verify(INDEX_DIR)
    print("✅ 완료")
    print(" - faiss.index :", index_path)
    print(" - stats.json  :", stats_path)
    print(" - global_ids  :", gid_path, f"({gid_len})")
    print(" - ntotal/dim  :", check['ntotal'], "/", check['dimension'])
    print(" - datasets    :", check['datasets'])
    if check["version_ranges"]:
        print(" - version_ranges :", check["version_ranges"][:3], "... (총", len(check["version_ranges"]), ")")

main()


✅ 완료
 - faiss.index : /content/drive/MyDrive/Faiss/Snowflake_snowflake_arctic_embed_l_v2.0/index/faiss.index
 - stats.json  : /content/drive/MyDrive/Faiss/Snowflake_snowflake_arctic_embed_l_v2.0/index/stats.json
 - global_ids  : /content/drive/MyDrive/Faiss/Snowflake_snowflake_arctic_embed_l_v2.0/index/global_ids.json (38730)
 - ntotal/dim  : 38730 / 1024
 - datasets    : ['2.4_Snowflake_snowflake_arctic_embed_l_v2.0_cls', '2.5_Snowflake_snowflake_arctic_embed_l_v2.0_cls', '2.6_Snowflake_snowflake_arctic_embed_l_v2.0_cls', '2.7_Snowflake_snowflake_arctic_embed_l_v2.0_cls', '2.8_Snowflake_snowflake_arctic_embed_l_v2.0_cls']
 - version_ranges : [{'version': '2.4_Snowflake_snowflake_arctic_embed_l_v2.0_cls', 'start': 0, 'end': 7901}, {'version': '2.5_Snowflake_snowflake_arctic_embed_l_v2.0_cls', 'start': 7902, 'end': 16023}, {'version': '2.6_Snowflake_snowflake_arctic_embed_l_v2.0_cls', 'start': 16024, 'end': 24116}] ... (총 5 )


- embeddings - index 매핑/중복 검사

In [37]:
# ==== CONFIG (검증 전용: 모델명만 바꿔 쓰면 됨) ====
from pathlib import Path
BASE_DIR   = Path("/content/drive/MyDrive/Faiss/Snowflake_snowflake_arctic_embed_l_v2.0")  # 경로 설정
EMB_DIR    = BASE_DIR / "embeddings"
INDEX_DIR  = BASE_DIR / "index"
MODEL_NAME = "Snowflake_snowflake_arctic_embed_l_v2.0_cls"  # _cls/_mean까지

# ==== IMPORTS ====
from collections import Counter
import json, numpy as np, faiss, os, glob

# === 1) stats.json / global_ids.json / faiss.index 정합성 확장검증 ===
with open(INDEX_DIR / "stats.json", encoding="utf-8") as f:
    stats = json.load(f)

with open(INDEX_DIR / "global_ids.json", encoding="utf-8") as f:
    gids = json.load(f)["ids"]

idx = faiss.read_index(str(INDEX_DIR / "faiss.index"))

print("=== 기본 정합성 ===")
print("ntotal == total_rows :", idx.ntotal == stats["total_rows"], idx.ntotal, stats["total_rows"])
print("dim == stats.dimension:", idx.d == stats["dimension"], idx.d, stats["dimension"])
print("len(global_ids) == total_rows:", len(gids) == stats["total_rows"], len(gids), stats["total_rows"])

# === 2) version_ranges별 길이 == datasets.rows 일치 여부 ===
print("\n=== version_ranges vs datasets.rows ===")
vr_ok = True
for vr in stats.get("version_ranges", []):
    version, s, e = vr["version"], vr["start"], vr["end"]
    expected = stats["datasets"][version]["rows"]
    actual = (e - s + 1)
    ok = (expected == actual)
    print(f"{version:>35} | start={s:>6}, end={e:>6}, rows={expected:>6} | span={actual:>6} | ok={ok}")
    vr_ok = vr_ok and ok
print("ALL_OK:", vr_ok)

# === 3) 샘플 검증: 각 버전의 앞/뒤 2개 글로벌 인덱스가 원본 id 순서와 일치하는지 ===
def key_of(path_str: str) -> str:
    """버전 키 통합 파서: 옛 e5 형식과 새 통일형식 모두 대응."""
    name = os.path.basename(path_str)

    # (A) 옛 e5 형식: id_mapping_torchdocs_<key>_intfloat...json
    if name.startswith("id_mapping_torchdocs_") and "_intfloat" in name:
        return name.split("id_mapping_torchdocs_")[1].split("_intfloat")[0]
    if name.startswith("id_mapping_torchdocs_") and "_mean" in name:
        return name.split("id_mapping_torchdocs_")[1].split("_mean")[0]

    # (B) 새 통일 형식: id_<ver>_<MODEL_NAME>[.json]
    sig = f"_{MODEL_NAME}"
    if name.startswith("id_") and sig in name:
        core = name[len("id_"):]
        # 끝 확장자 제거
        core = core.replace(".json", "")
        return core  # "<ver>_<MODEL_NAME>"

    return name  # 알 수 없으면 원본명(교집합에서 걸러짐)

# 새 통일 형식(+확장자 없는 케이스)과 e5 형식 모두 검색
map_paths = sorted(set(
    glob.glob(str(EMB_DIR / f"id_*_{MODEL_NAME}.json")) +
    glob.glob(str(EMB_DIR / f"id_*_{MODEL_NAME}")) +
    glob.glob(str(EMB_DIR / "id_mapping_torchdocs_*_e5_intfloat_e5_large_v2_mean.json"))
))
map_by_key = {key_of(p): p for p in map_paths}

print("\n=== 샘플 순서 일치 확인 (각 버전 앞/뒤 2개) ===")
for vr in stats.get("version_ranges", []):
    ver, s, e = vr["version"], vr["start"], vr["end"]
    if ver not in map_by_key:
        raise RuntimeError(f"[매핑 파일 미발견] version='{ver}' 를 키로 갖는 id 파일을 찾지 못했습니다.\n"
                           f"- 검색한 경로: {EMB_DIR}\n- MODEL_NAME='{MODEL_NAME}'\n- 사용 가능한 키 예: {sorted(map_by_key.keys())[:5]} ...")
    with open(map_by_key[ver], encoding="utf-8") as f:
        data = json.load(f)
    rows = data["id_mapping"] if isinstance(data, dict) and "id_mapping" in data else data
    if len(rows) == 0:
        print(ver, "빈 파일")
        continue

    picks = [0, 1, max(0, len(rows)-2), len(rows)-1] if len(rows) > 3 else list(range(len(rows)))
    ok_all = True
    for i in picks:
        gid = gids[s + i]
        rid = rows[i]["id"] if isinstance(rows[i], dict) else rows[i]
        ok = (gid == rid)
        ok_all = ok_all & ok
        print(f"{ver}  idx={i:>5} | global[{s+i}] == rows[{i}] ? {ok}")
    print(" -> version ok:", ok_all)

# === 4) 벡터 유효성(재구성 & L2 노름 ~ 1) 샘플 체크 ===
try:
    import numpy as _np
    N = min(10, idx.ntotal)
    ids = _np.random.choice(idx.ntotal, size=N, replace=False)
    bad = 0
    for i in ids:
        v = _np.empty((idx.d,), dtype='float32')
        idx.reconstruct(i, v)
        norm = float(_np.linalg.norm(v))
        if not (_np.isfinite(norm) and 0.99 <= norm <= 1.01):
            bad += 1
    print("\n=== 재구성 벡터 정규화 검사 ===")
    print(f"샘플 {N}개 중 정규화 범위(≈1) 벗어난 개수:", bad)
except Exception as e:
    print("\n(참고) 벡터 재구성 검사는 생략됨:", e)

# === 5) 글로벌 중복 id 존재 여부(정보용) ===
cnt = Counter(gids)
dup = [k for k, c in cnt.items() if c > 1]
print("\n=== global_ids 중복 id 개수 ===")
print(len(dup))
if dup:
    print("예시 5개:", dup[:5])


=== 기본 정합성 ===
ntotal == total_rows : True 38730 38730
dim == stats.dimension: True 1024 1024
len(global_ids) == total_rows: True 38730 38730

=== version_ranges vs datasets.rows ===
2.4_Snowflake_snowflake_arctic_embed_l_v2.0_cls | start=     0, end=  7901, rows=  7902 | span=  7902 | ok=True
2.5_Snowflake_snowflake_arctic_embed_l_v2.0_cls | start=  7902, end= 16023, rows=  8122 | span=  8122 | ok=True
2.6_Snowflake_snowflake_arctic_embed_l_v2.0_cls | start= 16024, end= 24116, rows=  8093 | span=  8093 | ok=True
2.7_Snowflake_snowflake_arctic_embed_l_v2.0_cls | start= 24117, end= 32319, rows=  8203 | span=  8203 | ok=True
2.8_Snowflake_snowflake_arctic_embed_l_v2.0_cls | start= 32320, end= 38729, rows=  6410 | span=  6410 | ok=True
ALL_OK: True

=== 샘플 순서 일치 확인 (각 버전 앞/뒤 2개) ===
2.4_Snowflake_snowflake_arctic_embed_l_v2.0_cls  idx=    0 | global[0] == rows[0] ? True
2.4_Snowflake_snowflake_arctic_embed_l_v2.0_cls  idx=    1 | global[1] == rows[1] ? True
2.4_Snowflake_snowflake_arctic_

- 중복 상세 확인

In [38]:
# === 중복 id 상세 & 벡터 동일성 확인 (출력만) ===
# 단일 스크립트: 옛 e5(torchdocs) 형식과 새 통일형식(embeddings_<ver>_<MODEL>.npy / id_<ver>_<MODEL>) 모두 지원
from pathlib import Path
from collections import Counter, defaultdict
import json, numpy as np, glob, os, faiss

# ==== CONFIG ====
BASE_DIR   = Path("/content/drive/MyDrive/Faiss/Snowflake_snowflake_arctic_embed_l_v2.0")
EMB_DIR    = BASE_DIR / "embeddings"
INDEX_DIR  = BASE_DIR / "index"
# 새 통일형식 사용 시 모델명 지정 (예: "BAAI_bge_base_en_cls", "sentence_transformers_all_MiniLM_L6_v2_mean", ...)
MODEL_NAME = "Snowflake_snowflake_arctic_embed_l_v2.0_cls"

# ==== 공통 유틸 ====
def key_of(path_str: str) -> str:
    """
    버전 키를 통일해서 반환.
      - 옛 e5 형식: id_mapping_torchdocs_<key>_intfloat...json -> <key> (예: '2.4_chunks_e5')
      - 새 통일형식: id_<ver>_<MODEL_NAME>[.json] -> '<ver>_<MODEL_NAME>'
    """
    name = os.path.basename(path_str)

    # (A) e5(torchdocs) 형식
    if name.startswith("id_mapping_torchdocs_") and "_intfloat" in name:
        return name.split("id_mapping_torchdocs_")[1].split("_intfloat")[0]
    if name.startswith("id_mapping_torchdocs_") and "_mean" in name:
        return name.split("id_mapping_torchdocs_")[1].split("_mean")[0]

    # (B) 새 통일형식
    if MODEL_NAME:
        sig = f"_{MODEL_NAME}"
        if name.startswith("id_") and sig in name:
            core = name[len("id_"):]
            core = core.replace(".json", "")
            return core  # "<ver>_<MODEL_NAME>"

    return name  # 알 수 없는 경우(교집합에서 걸러짐)

def load_ids(map_path: str):
    with open(map_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    id_list = data["id_mapping"] if isinstance(data, dict) and "id_mapping" in data else data
    if not isinstance(id_list, list) or not id_list or not isinstance(id_list[0], dict) or "id" not in id_list[0]:
        raise ValueError(f"[형식 오류] {map_path} : 'id' 키를 가진 dict 리스트가 필요합니다.")
    return id_list

def vec_path_from_version_key(version_key: str) -> str:
    """
    version_key에 맞는 임베딩 npy 경로를 생성.
      - e5(torchdocs): '2.4_chunks_e5' -> embeddings_torchdocs_<key>_intfloat_e5_large_v2_mean.npy
      - 새 통일형식:   '2.4_<MODEL_NAME>' -> embeddings_<ver>_<MODEL_NAME>.npy
    """
    if MODEL_NAME and version_key.endswith(MODEL_NAME):
        ver = version_key[:-(len(MODEL_NAME)+1)]  # 앞의 '<ver>_'만 추출
        return str(EMB_DIR / f"embeddings_{ver}_{MODEL_NAME}.npy")
    else:
        # 옛 e5(torchdocs)
        return str(EMB_DIR / f"embeddings_torchdocs_{version_key}_intfloat_e5_large_v2_mean.npy")

def load_vec(version_key: str, local_idx: int) -> np.ndarray:
    """해당 버전의 로컬 인덱스 벡터 한 행을 정규화하여 반환."""
    npy_path = vec_path_from_version_key(version_key)
    X = np.load(npy_path, mmap_mode="r")  # (rows, dim)
    v = X[local_idx].astype("float32")
    n = float(np.linalg.norm(v))
    return (v / n) if n > 0 else v

def cosine(a: np.ndarray, b: np.ndarray) -> float:
    return float(np.dot(a, b))  # a,b는 L2 정규화된 상태

# ==== 데이터 로드 ====
with open(INDEX_DIR / "stats.json", encoding="utf-8") as f:
    stats = json.load(f)
with open(INDEX_DIR / "global_ids.json", encoding="utf-8") as f:
    gids = json.load(f)["ids"]
idx = faiss.read_index(str(INDEX_DIR / "faiss.index"))

# id 매핑 파일 찾기(두 체계 모두 검색)
map_paths = sorted(set(
    glob.glob(str(EMB_DIR / "id_mapping_torchdocs_*_e5_intfloat_e5_large_v2_mean.json")) +
    (glob.glob(str(EMB_DIR / f"id_*_{MODEL_NAME}.json")) + glob.glob(str(EMB_DIR / f"id_*_{MODEL_NAME}")) if MODEL_NAME else [])
))
map_by_key = {key_of(p): p for p in map_paths}

# ==== 버전 범위 헬퍼 ====
vranges = stats.get("version_ranges", [])
def which_version(global_idx: int):
    for vr in vranges:
        if vr["start"] <= global_idx <= vr["end"]:
            return vr["version"], global_idx - vr["start"]
    return "unknown", -1

# ==== 1) 글로벌 중복 id 찾기 ====
cnt = Counter(gids)
dup_ids = [k for k, c in cnt.items() if c > 1]
print("=== 중복 id 개수 ===", len(dup_ids))

# ==== 2) 각 중복 id의 등장 위치 수집 ====
dup_locs = {}  # id -> [(version_key, global_idx, local_idx)]
for did in dup_ids:
    pos = [i for i, v in enumerate(gids) if v == did]
    locs = []
    for gi in pos:
        ver, local = which_version(gi)
        locs.append((ver, gi, local))
    dup_locs[did] = sorted(locs, key=lambda x: (x[0], x[1]))

# ==== 3) 벡터 동일성 확인 (코사인 ~ 1.0) ====
print("\n=== 중복 id 상세 (최대 12개 전부 표시) ===")
for did in dup_ids[:12]:
    locs = dup_locs[did]
    print(f"\n- id={did} (등장 {len(locs)}회)")
    for ver, gi, li in locs:
        print(f"  · {ver}: global[{gi}] / local[{li}]")
    # 벡터 비교
    base_v = load_vec(locs[0][0], locs[0][2])
    cos_all = []
    for ver, gi, li in locs[1:]:
        v = load_vec(ver, li)
        cos_all.append(cosine(base_v, v))
    if cos_all:
        print("  코사인 유사도(첫 벡터 vs 나머지):", [f"{c:.6f}" for c in cos_all])
        all_one = all(abs(c-1.0) < 1e-6 for c in cos_all)
        print("  → 모두 완전히 동일 벡터?", all_one)
    else:
        print("  (중복 한 쌍만 존재)")

print("\n=== 요약 ===")
only_within_version = all(len({v for v,_,_ in locs}) == 1 for locs in dup_locs.values())
print("모든 중복이 같은 버전 내에서만 발생?", only_within_version)


=== 중복 id 개수 === 12

=== 중복 id 상세 (최대 12개 전부 표시) ===

- id=300396de51e1f1034ce64f4e3a36b903f43bb985 (등장 2회)
  · 2.4_Snowflake_snowflake_arctic_embed_l_v2.0_cls: global[113] / local[113]
  · 2.4_Snowflake_snowflake_arctic_embed_l_v2.0_cls: global[117] / local[117]
  코사인 유사도(첫 벡터 vs 나머지): ['0.463110']
  → 모두 완전히 동일 벡터? False

- id=8907328a9ef69bc453f3f55e3fd90bd461e14678 (등장 2회)
  · 2.4_Snowflake_snowflake_arctic_embed_l_v2.0_cls: global[7451] / local[7451]
  · 2.4_Snowflake_snowflake_arctic_embed_l_v2.0_cls: global[7485] / local[7485]
  코사인 유사도(첫 벡터 vs 나머지): ['0.431984']
  → 모두 완전히 동일 벡터? False

- id=f18b3bccedd55f8221497387b64325c2ebf306ed (등장 2회)
  · 2.4_Snowflake_snowflake_arctic_embed_l_v2.0_cls: global[7576] / local[7576]
  · 2.4_Snowflake_snowflake_arctic_embed_l_v2.0_cls: global[7578] / local[7578]
  코사인 유사도(첫 벡터 vs 나머지): ['0.404683']
  → 모두 완전히 동일 벡터? False

- id=be917dc4de73f230d926d09a98c776060e88f966 (등장 2회)
  · 2.5_Snowflake_snowflake_arctic_embed_l_v2.0_cls: global[8018] / 